In [1]:
!pip install spotipy

   ---------------------------------------- 0.0/250.3 kB ? eta -:--:--
   ------------------- -------------------- 122.9/250.3 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 250.3/250.3 kB 2.6 MB/s eta 0:00:00


In [2]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
CLIENT_ID = '06dbc6bd9b654f9faffda333469fc2a8'
CLIENT_SECRET = '5a9c9cb226bb40a18f4ead88fd0eea98'
SPOTIPY_URL='https://accounts.spotify.com/api/token'

In [4]:
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
track_data = sp.search(q='track:'+'shape',type='track', limit= 50)

In [6]:
track_result = []
for i, item in enumerate(track_data['tracks']['items']):
    track = item['album']
    track_id = item['id']
    track_name = item['name']
    popularity = item['popularity']
    explicit = item['explicit']
    track_result.append((i, track['artists'][0]['name'], track['name'], track_id, track_name, track['release_date'], popularity, explicit))

In [7]:
track_df = pd.DataFrame(track_result, index=None, columns=('Item', 'Artist', 'Album Name', 'Id', 'Song Name', 'Release Date', 'Popularity', 'Explicit'))

In [8]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2022', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

In [9]:
track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})
print(track_df.shape)
track_df.head()

(1000, 6)


,id,track_name,track_popularity,explicit,artist_name,artist_id
0,1bDbXMyjaUIooNwFE9wn0N,Rich Flex,96,True,Drake,3TVXtAsR1Inumwj472S9r4
1,3OHfY25tqY28d16oZczHc8,Kill Bill,93,False,SZA,7tYKF4w9nC0nq9CsPZTHyP
2,3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),100,False,Sam Smith,2wY79sveU1sp5g7SokKOiI
3,0V3wPSX9ygBnCm8psDIegu,Anti-Hero,97,False,Taylor Swift,06HL4z0CvFAxyc27GXpf02
4,1xK59OXxi2TAAAbmZK0kBL,Romantic Homicide,94,False,d4vd,5y8tKLUfMvliMe8IKamR32


In [10]:
features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[0])
    features_df = pd.concat([features_df, local_features])
    
df_2022 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2022

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,1bDbXMyjaUIooNwFE9wn0N,Rich Flex,96.0,True,Drake,3TVXtAsR1Inumwj472S9r4,0.561,0.520,11.0,-9.342,...,0.3550,0.424,153.150,audio_features,spotify:track:1bDbXMyjaUIooNwFE9wn0N,https://api.spotify.com/v1/tracks/1bDbXMyjaUIo...,https://api.spotify.com/v1/audio-analysis/1bDb...,239360.0,3.0,NaN
1,3OHfY25tqY28d16oZczHc8,Kill Bill,93.0,False,SZA,7tYKF4w9nC0nq9CsPZTHyP,0.644,0.728,8.0,-5.750,...,0.1610,0.430,88.993,audio_features,spotify:track:3OHfY25tqY28d16oZczHc8,https://api.spotify.com/v1/tracks/3OHfY25tqY28...,https://api.spotify.com/v1/audio-analysis/3OHf...,153947.0,4.0,NaN
2,3nqQXoyQOWXiESFLlDF1hG,Unholy (feat. Kim Petras),100.0,False,Sam Smith,2wY79sveU1sp5g7SokKOiI,0.714,0.472,2.0,-7.375,...,0.2660,0.238,131.121,audio_features,spotify:track:3nqQXoyQOWXiESFLlDF1hG,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...,156943.0,4.0,NaN
3,0V3wPSX9ygBnCm8psDIegu,Anti-Hero,97.0,False,Taylor Swift,06HL4z0CvFAxyc27GXpf02,0.637,0.643,4.0,-6.571,...,0.1420,0.533,97.008,audio_features,spotify:track:0V3wPSX9ygBnCm8psDIegu,https://api.spotify.com/v1/tracks/0V3wPSX9ygBn...,https://api.spotify.com/v1/audio-analysis/0V3w...,200690.0,4.0,NaN
4,1xK59OXxi2TAAAbmZK0kBL,Romantic Homicide,94.0,False,d4vd,5y8tKLUfMvliMe8IKamR32,0.571,0.544,6.0,-10.613,...,0.3220,0.216,132.052,audio_features,spotify:track:1xK59OXxi2TAAAbmZK0kBL,https://api.spotify.com/v1/tracks/1xK59OXxi2TA...,https://api.spotify.com/v1/audio-analysis/1xK5...,132631.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,6LeyuBs6hub5NijWsKaMGN,Vaivén,0.0,False,Daddy Yankee,4VMYDCV2IEDYJArk749S6m,0.768,0.894,10.0,-2.482,...,0.0856,0.737,105.962,audio_features,spotify:track:6LeyuBs6hub5NijWsKaMGN,https://api.spotify.com/v1/tracks/6LeyuBs6hub5...,https://api.spotify.com/v1/audio-analysis/6Ley...,224333.0,4.0,NaN
998,49JKBLXIhUyLUCO0Lobzhy,RUMBATÓN,0.0,False,Daddy Yankee,4VMYDCV2IEDYJArk749S6m,0.685,0.909,3.0,-4.034,...,0.3500,0.922,95.004,audio_features,spotify:track:49JKBLXIhUyLUCO0Lobzhy,https://api.spotify.com/v1/tracks/49JKBLXIhUyL...,https://api.spotify.com/v1/audio-analysis/49JK...,248769.0,4.0,NaN
999,3g4FWv8og7I56g7wbo0ymX,La Noche De Los Dos,0.0,False,Daddy Yankee,4VMYDCV2IEDYJArk749S6m,0.762,0.790,9.0,-3.944,...,0.1720,0.748,129.988,audio_features,spotify:track:3g4FWv8og7I56g7wbo0ymX,https://api.spotify.com/v1/tracks/3g4FWv8og7I5...,https://api.spotify.com/v1/audio-analysis/3g4F...,223360.0,4.0,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_2022.to_csv('df_2022.csv')

In [11]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2021', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])
        
track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2021 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2021

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,3cBsEDNhFI9E82vPj3kvi3,Wasted On You,84.0,False,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,0.505,0.657,11.0,-5.240,...,0.1260,0.252,196.000,audio_features,spotify:track:3cBsEDNhFI9E82vPj3kvi3,https://api.spotify.com/v1/tracks/3cBsEDNhFI9E...,https://api.spotify.com/v1/audio-analysis/3cBs...,178520.0,3.0,NaN
1,3azJifCSqg9fRij2yKIbWz,The Color Violet,87.0,False,Tory Lanez,2jku7tDXc6XoB6MO2hFuqg,0.645,0.534,6.0,-10.800,...,0.0863,0.463,105.020,audio_features,spotify:track:3azJifCSqg9fRij2yKIbWz,https://api.spotify.com/v1/tracks/3azJifCSqg9f...,https://api.spotify.com/v1/audio-analysis/3azJ...,226467.0,4.0,NaN
2,2BcMwX1MPV6ZHP4tUT9uq6,Knife Talk (with 21 Savage ft. Project Pat),84.0,True,Drake,3TVXtAsR1Inumwj472S9r4,0.849,0.424,5.0,-9.579,...,0.0834,0.153,145.887,audio_features,spotify:track:2BcMwX1MPV6ZHP4tUT9uq6,https://api.spotify.com/v1/tracks/2BcMwX1MPV6Z...,https://api.spotify.com/v1/audio-analysis/2BcM...,242966.0,4.0,NaN
3,4h4QlmocP3IuwYEj2j14p8,Sunroof,89.0,False,Nicky Youre,7qmpXeNz2ojlMl2EEfkeLs,0.768,0.714,10.0,-5.110,...,0.1500,0.842,131.443,audio_features,spotify:track:4h4QlmocP3IuwYEj2j14p8,https://api.spotify.com/v1/tracks/4h4QlmocP3Iu...,https://api.spotify.com/v1/audio-analysis/4h4Q...,163026.0,4.0,NaN
4,65OVbaJR5O1RmwOQx0875b,Wants and Needs (feat. Lil Baby),83.0,True,Drake,3TVXtAsR1Inumwj472S9r4,0.578,0.449,1.0,-6.349,...,0.1190,0.100,136.006,audio_features,spotify:track:65OVbaJR5O1RmwOQx0875b,https://api.spotify.com/v1/tracks/65OVbaJR5O1R...,https://api.spotify.com/v1/audio-analysis/65OV...,192956.0,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,7oe04qLMdQxHXhcGIfShXf,Piggypink,65.0,False,Hotel Ugly,35WVTyRnKAoaGExqgktVyb,0.877,0.462,0.0,-14.419,...,0.1450,0.888,119.989,audio_features,spotify:track:7oe04qLMdQxHXhcGIfShXf,https://api.spotify.com/v1/tracks/7oe04qLMdQxH...,https://api.spotify.com/v1/audio-analysis/7oe0...,160000.0,4.0,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_2021.to_csv('df_2021.csv')

In [12]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2020', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2020 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2020

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,3hxIUxnT27p5WcmjGUXNwx,Shut up My Moms Calling,88.0,False,Hotel Ugly,35WVTyRnKAoaGExqgktVyb,0.485,0.409,9.0,-10.711,...,0.1030,0.376,138.419,audio_features,spotify:track:3hxIUxnT27p5WcmjGUXNwx,https://api.spotify.com/v1/tracks/3hxIUxnT27p5...,https://api.spotify.com/v1/audio-analysis/3hxI...,164571.0,4.0,NaN
1,2J6OF7CkpdQGSfm1wdclqn,Clouded,84.0,True,Brent Faiyaz,3tlXnStJ1fFhdScmQeLpuG,0.583,0.344,2.0,-8.765,...,0.1430,0.581,65.203,audio_features,spotify:track:2J6OF7CkpdQGSfm1wdclqn,https://api.spotify.com/v1/tracks/2J6OF7CkpdQG...,https://api.spotify.com/v1/audio-analysis/2J6O...,110129.0,3.0,NaN
2,6pcywuOeGGWeOQzdUyti6k,Glock In My Lap,84.0,True,21 Savage,1URnnhqYAYcrqrcwql10ft,0.847,0.733,10.0,-6.439,...,0.1510,0.202,130.029,audio_features,spotify:track:6pcywuOeGGWeOQzdUyti6k,https://api.spotify.com/v1/tracks/6pcywuOeGGWe...,https://api.spotify.com/v1/audio-analysis/6pcy...,193591.0,4.0,NaN
3,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights,91.0,False,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0.514,0.730,1.0,-5.934,...,0.0897,0.334,171.005,audio_features,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040.0,4.0,NaN
4,4xqrdfXkTW4T0RauPLv3WA,Heather,89.0,False,Conan Gray,4Uc8Dsxct0oMqx0P6i60ea,0.357,0.425,5.0,-7.301,...,0.3220,0.270,102.078,audio_features,spotify:track:4xqrdfXkTW4T0RauPLv3WA,https://api.spotify.com/v1/tracks/4xqrdfXkTW4T...,https://api.spotify.com/v1/audio-analysis/4xqr...,198040.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,4JuYtPKJJk0HobzktzN4Qg,damn!,63.0,True,Jeris Johnson,2hmePXeTr2b7cdRAtRjvPq,0.754,0.591,0.0,-7.341,...,0.2090,0.859,119.960,audio_features,spotify:track:4JuYtPKJJk0HobzktzN4Qg,https://api.spotify.com/v1/tracks/4JuYtPKJJk0H...,https://api.spotify.com/v1/audio-analysis/4JuY...,114250.0,4.0,NaN
997,5JWPUEov2wlX7c0jhYZpeB,Lost In Yesterday,67.0,False,Tame Impala,5INjqkS1o8h1imAzPqGZBb,0.569,0.840,1.0,-4.123,...,0.0995,0.749,183.059,audio_features,spotify:track:5JWPUEov2wlX7c0jhYZpeB,https://api.spotify.com/v1/tracks/5JWPUEov2wlX...,https://api.spotify.com/v1/audio-analysis/5JWP...,249587.0,3.0,NaN
998,5E7p95xh9Ul4vZp4LEoJwi,Tick Tock,68.0,False,Joji,3MZsBdqDrRTJihTHQrO6Dq,0.781,0.613,4.0,-5.582,...,0.0976,0.613,80.530,audio_features,spotify:track:5E7p95xh9Ul4vZp4LEoJwi,https://api.spotify.com/v1/tracks/5E7p95xh9Ul4...,https://api.spotify.com/v1/audio-analysis/5E7p...,132215.0,4.0,NaN
999,3Uo7WG0vmLQ07WB4BDwy7D,willow,69.0,False,Taylor Swift,06HL4z0CvFAxyc27GXpf02,0.392,0.579,7.0,-9.195,...,0.1450,0.549,80.961,audio_features,spotify:track:3Uo7WG0vmLQ07WB4BDwy7D,https://api.spotify.com/v1/tracks/3Uo7WG0vmLQ0...,https://api.spotify.com/v1/audio-analysis/3Uo7...,214707.0,4.0,NaN


In [ ]:
df_2020.to_csv('df_2020.csv')

In [13]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2019 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2019

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5IgjP7X4th6nMNDh4akUHb,Under The Influence,95,True,Chris Brown,7bXgB6jMjp9ATFy66eO08Z,0.733,0.690,9,-5.529,...,0.000001,0.105,0.310,116.992,audio_features,spotify:track:5IgjP7X4th6nMNDh4akUHb,https://api.spotify.com/v1/tracks/5IgjP7X4th6n...,https://api.spotify.com/v1/audio-analysis/5Igj...,184613,4
1,58k32my5lKofeZRtIvBDg9,HONEST,83,True,Baby Keem,5SXuuuRpukkTvsLuUknva1,0.676,0.536,2,-8.790,...,0.000034,0.105,0.373,79.037,audio_features,spotify:track:58k32my5lKofeZRtIvBDg9,https://api.spotify.com/v1/tracks/58k32my5lKof...,https://api.spotify.com/v1/audio-analysis/58k3...,172672,4
2,2Dct3GykKZ58hpWRFfe2Qd,Heading South,81,True,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,0.680,0.246,4,-14.112,...,0.000000,0.106,0.388,110.230,audio_features,spotify:track:2Dct3GykKZ58hpWRFfe2Qd,https://api.spotify.com/v1/tracks/2Dct3GykKZ58...,https://api.spotify.com/v1/audio-analysis/2Dct...,171693,4
3,2mvabkN1i2gLnGAPUVdwek,Christmas Tree Farm,84,False,Taylor Swift,06HL4z0CvFAxyc27GXpf02,0.598,0.680,7,-4.505,...,0.000000,0.324,0.437,134.002,audio_features,spotify:track:2mvabkN1i2gLnGAPUVdwek,https://api.spotify.com/v1/tracks/2mvabkN1i2gL...,https://api.spotify.com/v1/audio-analysis/2mva...,228027,4
4,51Grh1RyUDcMBbpuyUIUHI,rises the moon,84,False,Liana Flores,5zVu34ozw0BzXIcNHqkO0u,0.483,0.129,5,-20.149,...,0.145000,0.106,0.203,137.350,audio_features,spotify:track:51Grh1RyUDcMBbpuyUIUHI,https://api.spotify.com/v1/tracks/51Grh1RyUDcM...,https://api.spotify.com/v1/audio-analysis/51Gr...,161929,3


In [31]:
df_2019.to_csv('df_2019.csv')

In [14]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2018 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2018.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,7uoFMmxln0GPXQ0AcCBXRq,Snowman,93.0,False,Sia,5WUlDfRSoLAfcVSX1WnrxN,0.716,0.512,1.0,-6.257,...,0.0928,0.326,104.957,audio_features,spotify:track:7uoFMmxln0GPXQ0AcCBXRq,https://api.spotify.com/v1/tracks/7uoFMmxln0GP...,https://api.spotify.com/v1/audio-analysis/7uoF...,165907.0,3.0,NaN
1,6foY66mWZN0pSRjZ408c00,Whiskey Glasses,81.0,False,Morgan Wallen,4oUHIQIBe0LHzYfvXNW4QM,0.614,0.680,6.0,-4.580,...,0.1150,0.707,149.959,audio_features,spotify:track:6foY66mWZN0pSRjZ408c00,https://api.spotify.com/v1/tracks/6foY66mWZN0p...,https://api.spotify.com/v1/audio-analysis/6foY...,234347.0,4.0,NaN
2,1cKHdTo9u0ZymJdPGSh6nq,I Was Never There,91.0,False,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0.323,0.737,10.0,-4.008,...,0.1630,0.155,113.260,audio_features,spotify:track:1cKHdTo9u0ZymJdPGSh6nq,https://api.spotify.com/v1/tracks/1cKHdTo9u0Zy...,https://api.spotify.com/v1/audio-analysis/1cKH...,241067.0,4.0,NaN
3,78QR3Wp35dqAhFEc2qAGjE,Drip Too Hard (Lil Baby & Gunna),85.0,True,Lil Baby,5f7VJjfbwm532GiveGC0ZK,0.897,0.662,1.0,-6.903,...,0.5340,0.389,112.511,audio_features,spotify:track:78QR3Wp35dqAhFEc2qAGjE,https://api.spotify.com/v1/tracks/78QR3Wp35dqA...,https://api.spotify.com/v1/audio-analysis/78QR...,145543.0,4.0,NaN
4,4VXIryQMWpIdGgYR4TrjT1,All Girls Are The Same,85.0,True,Juice WRLD,4MCBfE4596Uoi2O4DtmEMz,0.671,0.529,0.0,-7.226,...,0.0856,0.203,161.989,audio_features,spotify:track:4VXIryQMWpIdGgYR4TrjT1,https://api.spotify.com/v1/tracks/4VXIryQMWpId...,https://api.spotify.com/v1/audio-analysis/4VXI...,165820.0,4.0,NaN


In [32]:
df_2018.to_csv('df_2018.csv')

In [15]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2017', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2017 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2017.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,3EaJDYHA0KnX88JvDhL9oa,Dark Red,89.0,False,Steve Lacy,57vWImR43h4CaDao012Ofp,0.603,0.784,6.0,-4.023,...,0.119,0.769,172.041,audio_features,spotify:track:3EaJDYHA0KnX88JvDhL9oa,https://api.spotify.com/v1/tracks/3EaJDYHA0KnX...,https://api.spotify.com/v1/audio-analysis/3EaJ...,173104.0,4.0,NaN
1,3AVrVz5rK8Hrqo9YGiVGN5,Apocalypse,84.0,False,Cigarettes After Sex,1QAJqy2dA3ihHBFIHRphZj,0.369,0.467,5.0,-9.018,...,0.109,0.174,94.473,audio_features,spotify:track:3AVrVz5rK8Hrqo9YGiVGN5,https://api.spotify.com/v1/tracks/3AVrVz5rK8Hr...,https://api.spotify.com/v1/audio-analysis/3AVr...,290147.0,4.0,NaN
2,5TXDeTFVRVY7Cvt0Dw4vWW,Revenge,88.0,True,XXXTENTACION,15UsOTVnJzReFVN1VCnxy4,0.746,0.251,11.0,-16.169,...,0.106,0.180,139.999,audio_features,spotify:track:5TXDeTFVRVY7Cvt0Dw4vWW,https://api.spotify.com/v1/tracks/5TXDeTFVRVY7...,https://api.spotify.com/v1/audio-analysis/5TXD...,120027.0,4.0,NaN
3,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),85.0,True,"Tyler, The Creator",4V8LLVI7PbaPR0K2TGSxFF,0.558,0.559,6.0,-9.222,...,0.109,0.620,78.558,audio_features,spotify:track:7KA4W4McWYRpgf0fWsJZWB,https://api.spotify.com/v1/tracks/7KA4W4McWYRp...,https://api.spotify.com/v1/audio-analysis/7KA4...,180387.0,4.0,NaN
4,5BbdKBZO0TH0GhfxUfyhL9,Freestyle,83.0,True,Lil Baby,5f7VJjfbwm532GiveGC0ZK,0.877,0.517,2.0,-5.426,...,0.143,0.255,119.996,audio_features,spotify:track:5BbdKBZO0TH0GhfxUfyhL9,https://api.spotify.com/v1/tracks/5BbdKBZO0TH0...,https://api.spotify.com/v1/audio-analysis/5Bbd...,162053.0,4.0,NaN


In [33]:
df_2017.to_csv('df_2017.csv')

In [16]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2016', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2016 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2016.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,2Ch7LmS7r2Gy2kc64wv3Bz,Die For You,89.0,False,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0.582,0.525,1.0,-7.163,...,0.1340,0.510,133.249,audio_features,spotify:track:2Ch7LmS7r2Gy2kc64wv3Bz,https://api.spotify.com/v1/tracks/2Ch7LmS7r2Gy...,https://api.spotify.com/v1/audio-analysis/2Ch7...,260253.0,4.0,NaN
1,3xKsf9qdS1CyvXSMEid6g8,Pink + White,86.0,False,Frank Ocean,2h93pZq0e7k5yf4dywlkpM,0.545,0.545,9.0,-7.362,...,0.4170,0.549,159.940,audio_features,spotify:track:3xKsf9qdS1CyvXSMEid6g8,https://api.spotify.com/v1/tracks/3xKsf9qdS1Cy...,https://api.spotify.com/v1/audio-analysis/3xKs...,184516.0,3.0,NaN
2,2LBqCSwhJGcFQeTHMVGwy3,Die For You,89.0,False,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0.586,0.525,1.0,-7.163,...,0.1340,0.508,133.629,audio_features,spotify:track:2LBqCSwhJGcFQeTHMVGwy3,https://api.spotify.com/v1/tracks/2LBqCSwhJGcF...,https://api.spotify.com/v1/audio-analysis/2LBq...,260253.0,4.0,NaN
3,4AFsRbaLKRWo3dDtjDFA2V,Me and Your Mama,85.0,False,Childish Gambino,73sIBHcqh3Z3NyqHKZ7FOL,0.545,0.442,5.0,-12.111,...,0.0664,0.242,118.073,audio_features,spotify:track:4AFsRbaLKRWo3dDtjDFA2V,https://api.spotify.com/v1/tracks/4AFsRbaLKRWo...,https://api.spotify.com/v1/audio-analysis/4AFs...,379227.0,3.0,NaN
4,7MXVkk9YMctZqd1Srtv4MB,Starboy,91.0,True,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0.679,0.587,7.0,-7.015,...,0.1370,0.486,186.003,audio_features,spotify:track:7MXVkk9YMctZqd1Srtv4MB,https://api.spotify.com/v1/tracks/7MXVkk9YMctZ...,https://api.spotify.com/v1/audio-analysis/7MXV...,230453.0,4.0,NaN


In [34]:
df_2016.to_csv('df_2016.csv')

In [17]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2015', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2015 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2015.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,7H0ya83CMmgFcOhw0UB6ow,Space Song,82.0,False,Beach House,56ZTgzPBDge0OvCGgMO3OY,0.508,0.792,0.0,-7.311,...,0.1450,0.601,147.067,audio_features,spotify:track:7H0ya83CMmgFcOhw0UB6ow,https://api.spotify.com/v1/tracks/7H0ya83CMmgF...,https://api.spotify.com/v1/audio-analysis/7H0y...,320467.0,4.0,NaN
1,7fBv7CLKzipRk6EC6TWHOB,The Hills,88.0,True,The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,0.585,0.564,0.0,-7.063,...,0.1350,0.137,113.003,audio_features,spotify:track:7fBv7CLKzipRk6EC6TWHOB,https://api.spotify.com/v1/tracks/7fBv7CLKzipR...,https://api.spotify.com/v1/audio-analysis/7fBv...,242253.0,4.0,NaN
2,3pXF1nA74528Edde4of9CC,Don't,84.0,True,Bryson Tiller,2EMAnMvWE2eb56ToJVfCWs,0.765,0.356,11.0,-5.556,...,0.0963,0.189,96.991,audio_features,spotify:track:3pXF1nA74528Edde4of9CC,https://api.spotify.com/v1/tracks/3pXF1nA74528...,https://api.spotify.com/v1/audio-analysis/3pXF...,198293.0,4.0,NaN
3,6Ox1rvlwEpB49drasQm6RF,Star Shopping,86.0,True,Lil Peep,2kCcBybjl3SAtIcwdWpUe3,0.538,0.471,4.0,-9.840,...,0.1160,0.307,93.070,audio_features,spotify:track:6Ox1rvlwEpB49drasQm6RF,https://api.spotify.com/v1/tracks/6Ox1rvlwEpB4...,https://api.spotify.com/v1/audio-analysis/6Ox1...,142000.0,4.0,NaN
4,3fqwjXwUGN6vbzIwvyFMhx,Tennessee Whiskey,81.0,False,Chris Stapleton,4YLtscXsxbVgi031ovDDdh,0.392,0.370,9.0,-10.888,...,0.0821,0.512,48.718,audio_features,spotify:track:3fqwjXwUGN6vbzIwvyFMhx,https://api.spotify.com/v1/tracks/3fqwjXwUGN6v...,https://api.spotify.com/v1/audio-analysis/3fqw...,293293.0,4.0,NaN


In [35]:
df_2015.to_csv('df_2015.csv')

In [18]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2014', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2014 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2014.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,0lizgQ7Qw35od7CYaoMBZb,Santa Tell Me,95,False,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,0.525,0.621,7,-7.364,...,0.000,0.2940,0.591,191.900,audio_features,spotify:track:0lizgQ7Qw35od7CYaoMBZb,https://api.spotify.com/v1/tracks/0lizgQ7Qw35o...,https://api.spotify.com/v1/audio-analysis/0liz...,204093,4
1,68Dni7IE4VyPkTOH9mRWHr,No Role Modelz,89,True,J. Cole,6l3HvQ5sa6mXTsMTB19rO5,0.690,0.521,10,-8.492,...,0.000,0.0534,0.494,100.028,audio_features,spotify:track:68Dni7IE4VyPkTOH9mRWHr,https://api.spotify.com/v1/tracks/68Dni7IE4VyP...,https://api.spotify.com/v1/audio-analysis/68Dn...,292799,4
2,4tqcoej1zPvwePZCzuAjJd,Wet Dreamz,85,True,J. Cole,6l3HvQ5sa6mXTsMTB19rO5,0.495,0.722,6,-8.108,...,0.000,0.1130,0.539,175.627,audio_features,spotify:track:4tqcoej1zPvwePZCzuAjJd,https://api.spotify.com/v1/tracks/4tqcoej1zPvw...,https://api.spotify.com/v1/audio-analysis/4tqc...,239267,4
3,4SqWKzw0CbA05TGszDgMlc,I Love You So,89,False,The Walters,027TpXKGwdXP7iwbjUSpV8,0.575,0.648,10,-4.891,...,0.000,0.1150,0.466,75.977,audio_features,spotify:track:4SqWKzw0CbA05TGszDgMlc,https://api.spotify.com/v1/tracks/4SqWKzw0CbA0...,https://api.spotify.com/v1/audio-analysis/4SqW...,160240,4
4,2LKOHdMsL0K9KwcPRlJK2v,After Dark,86,False,Mr.Kitty,0pWwt5vGNzezEhfAcc420Y,0.585,0.595,8,-10.444,...,0.266,0.0837,0.039,140.037,audio_features,spotify:track:2LKOHdMsL0K9KwcPRlJK2v,https://api.spotify.com/v1/tracks/2LKOHdMsL0K9...,https://api.spotify.com/v1/audio-analysis/2LKO...,259147,4


In [36]:
df_2014.to_csv('df_2014.csv')

In [19]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2013', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2013 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2013.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3YZE5qDV7u1ZD1gZc47ZeR,Underneath the Tree,85,False,Kelly Clarkson,3BmGtnKgCSGYIUhmivXKWX,0.509,0.812,8,-5.439,...,0.00002,0.2140,0.689,159.878,audio_features,spotify:track:3YZE5qDV7u1ZD1gZc47ZeR,https://api.spotify.com/v1/tracks/3YZE5qDV7u1Z...,https://api.spotify.com/v1/audio-analysis/3YZE...,229640,4
1,2QjOHCTQ1Jl3zawyYOpxh6,Sweater Weather,93,False,The Neighbourhood,77SW9BnxLY8rJ0RciFqkHh,0.612,0.807,10,-2.810,...,0.01770,0.1010,0.398,124.053,audio_features,spotify:track:2QjOHCTQ1Jl3zawyYOpxh6,https://api.spotify.com/v1/tracks/2QjOHCTQ1Jl3...,https://api.spotify.com/v1/audio-analysis/2QjO...,240400,4
2,3sNVsP50132BTNlImLx70i,Bound 2,86,True,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,0.367,0.665,1,-2.821,...,0.00000,0.1130,0.310,148.913,audio_features,spotify:track:3sNVsP50132BTNlImLx70i,https://api.spotify.com/v1/tracks/3sNVsP50132B...,https://api.spotify.com/v1/audio-analysis/3sNV...,229147,4
3,5XeFesFbtLpXzIVDNQP22n,I Wanna Be Yours,93,False,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,0.464,0.417,0,-9.345,...,0.02200,0.0974,0.479,67.528,audio_features,spotify:track:5XeFesFbtLpXzIVDNQP22n,https://api.spotify.com/v1/tracks/5XeFesFbtLpX...,https://api.spotify.com/v1/audio-analysis/5XeF...,183956,4
4,79MSEdtXuudhGhC5AtG07g,Break from Toronto,84,True,PARTYNEXTDOOR,2HPaUgqeutzr3jx5a9WyDV,0.596,0.678,9,-5.180,...,0.00400,0.4180,0.259,117.066,audio_features,spotify:track:79MSEdtXuudhGhC5AtG07g,https://api.spotify.com/v1/tracks/79MSEdtXuudh...,https://api.spotify.com/v1/audio-analysis/79MS...,99213,4


In [37]:
df_2013.to_csv('df_2013.csv')

In [20]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2012', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2012 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2012.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5a1iz510sv2W9Dt1MvFd5R,It's Beginning to Look a Lot like Christmas,92,False,Michael Bublé,1GxkXlMwML1oSg5eLPiAz3,0.339,0.214,4,-11.714,...,0.000007,0.341,0.363,94.775,audio_features,spotify:track:5a1iz510sv2W9Dt1MvFd5R,https://api.spotify.com/v1/tracks/5a1iz510sv2W...,https://api.spotify.com/v1/audio-analysis/5a1i...,206640,3
1,2HbKqm4o0w5wEeEFXm2sD4,Money Trees,81,True,Kendrick Lamar,2YZyLoL8N0Wb9xBt1NhZWg,0.716,0.531,7,-7.355,...,0.000000,0.224,0.344,71.994,audio_features,spotify:track:2HbKqm4o0w5wEeEFXm2sD4,https://api.spotify.com/v1/tracks/2HbKqm4o0w5w...,https://api.spotify.com/v1/audio-analysis/2HbK...,386907,4
2,4YhY2qHdBmSX33uj8ms0oq,O Tannenbaum,81,False,Vince Guaraldi Trio,4ytkhMSAnrDP8XzRNlw9FS,0.583,0.191,5,-15.938,...,0.008240,0.126,0.342,123.303,audio_features,spotify:track:4YhY2qHdBmSX33uj8ms0oq,https://api.spotify.com/v1/tracks/4YhY2qHdBmSX...,https://api.spotify.com/v1/audio-analysis/4YhY...,309387,4
3,3GZD6HmiNUhxXYf8Gch723,Lost,85,True,Frank Ocean,2h93pZq0e7k5yf4dywlkpM,0.913,0.603,8,-4.892,...,0.000503,0.167,0.497,123.061,audio_features,spotify:track:3GZD6HmiNUhxXYf8Gch723,https://api.spotify.com/v1/tracks/3GZD6HmiNUhx...,https://api.spotify.com/v1/audio-analysis/3GZD...,234093,4
4,01Lr5YepbgjXAWR9iOEyH1,Love Sosa,82,True,Chief Keef,15iVAtD3s3FsQR4w1v6M0P,0.772,0.413,0,-8.193,...,0.000000,0.111,0.211,131.965,audio_features,spotify:track:01Lr5YepbgjXAWR9iOEyH1,https://api.spotify.com/v1/tracks/01Lr5YepbgjX...,https://api.spotify.com/v1/audio-analysis/01Lr...,246213,4


In [38]:
df_2012.to_csv('df_2012.csv')

In [21]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2011', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2011 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2011.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,7xapw9Oy21WpfEcib2ErSA,Mistletoe,91,False,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,0.657,0.560,6,-9.049,...,0.000000,0.0794,0.854,161.993,audio_features,spotify:track:7xapw9Oy21WpfEcib2ErSA,https://api.spotify.com/v1/tracks/7xapw9Oy21Wp...,https://api.spotify.com/v1/audio-analysis/7xap...,182947,4
1,6tjituizSxwSmBB5vtgHZE,Holly Jolly Christmas,83,False,Michael Bublé,1GxkXlMwML1oSg5eLPiAz3,0.647,0.465,7,-8.308,...,0.000000,0.0882,0.699,151.251,audio_features,spotify:track:6tjituizSxwSmBB5vtgHZE,https://api.spotify.com/v1/tracks/6tjituizSxwS...,https://api.spotify.com/v1/audio-analysis/6tji...,119787,4
2,0lLdorYw7lVrJydTINhWdI,It's Beginning to Look a Lot like Christmas,81,False,Michael Bublé,1GxkXlMwML1oSg5eLPiAz3,0.339,0.214,4,-11.714,...,0.000007,0.3410,0.363,94.775,audio_features,spotify:track:0lLdorYw7lVrJydTINhWdI,https://api.spotify.com/v1/tracks/0lLdorYw7lVr...,https://api.spotify.com/v1/audio-analysis/0lLd...,206640,3
3,1vZKP9XURuqMp1SpXGnoyb,Santa Baby,81,False,Eartha Kitt,1AwO9pWEBSBoWdEZu28XDC,0.704,0.103,5,-15.056,...,0.000000,0.1920,0.468,84.475,audio_features,spotify:track:1vZKP9XURuqMp1SpXGnoyb,https://api.spotify.com/v1/tracks/1vZKP9XURuqM...,https://api.spotify.com/v1/audio-analysis/1vZK...,206507,4
4,11BKm0j4eYoCPPpCONAVwA,Bloody Mary,80,False,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,0.591,0.637,9,-6.365,...,0.000002,0.1130,0.432,99.998,audio_features,spotify:track:11BKm0j4eYoCPPpCONAVwA,https://api.spotify.com/v1/tracks/11BKm0j4eYoC...,https://api.spotify.com/v1/audio-analysis/11BK...,244760,4


In [39]:
df_2011.to_csv('df_2011.csv')

In [26]:
df_2011

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,7xapw9Oy21WpfEcib2ErSA,Mistletoe,91,False,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,0.657,0.560,6,-9.049,...,0.000000,0.0794,0.854,161.993,audio_features,spotify:track:7xapw9Oy21WpfEcib2ErSA,https://api.spotify.com/v1/tracks/7xapw9Oy21Wp...,https://api.spotify.com/v1/audio-analysis/7xap...,182947,4
1,6tjituizSxwSmBB5vtgHZE,Holly Jolly Christmas,83,False,Michael Bublé,1GxkXlMwML1oSg5eLPiAz3,0.647,0.465,7,-8.308,...,0.000000,0.0882,0.699,151.251,audio_features,spotify:track:6tjituizSxwSmBB5vtgHZE,https://api.spotify.com/v1/tracks/6tjituizSxwS...,https://api.spotify.com/v1/audio-analysis/6tji...,119787,4
2,0lLdorYw7lVrJydTINhWdI,It's Beginning to Look a Lot like Christmas,81,False,Michael Bublé,1GxkXlMwML1oSg5eLPiAz3,0.339,0.214,4,-11.714,...,0.000007,0.3410,0.363,94.775,audio_features,spotify:track:0lLdorYw7lVrJydTINhWdI,https://api.spotify.com/v1/tracks/0lLdorYw7lVr...,https://api.spotify.com/v1/audio-analysis/0lLd...,206640,3
3,1vZKP9XURuqMp1SpXGnoyb,Santa Baby,81,False,Eartha Kitt,1AwO9pWEBSBoWdEZu28XDC,0.704,0.103,5,-15.056,...,0.000000,0.1920,0.468,84.475,audio_features,spotify:track:1vZKP9XURuqMp1SpXGnoyb,https://api.spotify.com/v1/tracks/1vZKP9XURuqM...,https://api.spotify.com/v1/audio-analysis/1vZK...,206507,4
4,11BKm0j4eYoCPPpCONAVwA,Bloody Mary,80,False,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,0.591,0.637,9,-6.365,...,0.000002,0.1130,0.432,99.998,audio_features,spotify:track:11BKm0j4eYoCPPpCONAVwA,https://api.spotify.com/v1/tracks/11BKm0j4eYoC...,https://api.spotify.com/v1/audio-analysis/11BK...,244760,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7dPxocEuvcOA0YC7woHPvu,Di Que Regresarás,56,False,La Original Banda El Limón de Salvador Lizárraga,2ghByd8ucnRTWceSAnAZ0G,0.612,0.584,0,-6.781,...,0.001450,0.1010,0.655,97.496,audio_features,spotify:track:7dPxocEuvcOA0YC7woHPvu,https://api.spotify.com/v1/tracks/7dPxocEuvcOA...,https://api.spotify.com/v1/audio-analysis/7dPx...,225147,3
996,1CruCvvFpvyiefcWYf0lEj,resonance,52,False,T.M.Revolution,3jdzcmtw5XOmOkSb2mRDtr,0.360,0.984,1,-3.156,...,0.000000,0.3010,0.503,164.935,audio_features,spotify:track:1CruCvvFpvyiefcWYf0lEj,https://api.spotify.com/v1/tracks/1CruCvvFpvyi...,https://api.spotify.com/v1/audio-analysis/1Cru...,224760,4
997,4sj6Hk441YK66GxLM6M5Iz,"If You Think This Song Is About You, It Probab...",50,False,Destroy Rebuild Until God Shows,6XjXff8FNS5CqpF8dVAtoo,0.597,0.970,4,-6.303,...,0.000014,0.3540,0.352,121.971,audio_features,spotify:track:4sj6Hk441YK66GxLM6M5Iz,https://api.spotify.com/v1/tracks/4sj6Hk441YK6...,https://api.spotify.com/v1/audio-analysis/4sj6...,150200,4
998,6toBkNzxa4oCYTipbAaaJ3,Start of Something Good,55,False,Daughtry,5P5FTygHyx2G57oszR3Wot,0.424,0.680,8,-4.311,...,0.000000,0.2240,0.415,157.867,audio_features,spotify:track:6toBkNzxa4oCYTipbAaaJ3,https://api.spotify.com/v1/tracks/6toBkNzxa4oC...,https://api.spotify.com/v1/audio-analysis/6toB...,264853,4


In [22]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2010', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2010 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2010.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,27RYrbL6S02LNVhDWVl38b,Happy Xmas (War Is Over) - Remastered 2010,91,False,John Lennon,4x1nvY2FN8jxqAFA0DA02H,0.321,0.640,2,-10.023,...,0.000000,0.7180,0.414,146.172,audio_features,spotify:track:27RYrbL6S02LNVhDWVl38b,https://api.spotify.com/v1/tracks/27RYrbL6S02L...,https://api.spotify.com/v1/audio-analysis/27RY...,214333,3
1,0JXXNGljqupsJaZsgSbMZV,Sure Thing,87,False,Miguel,360IAlyVv4PCEVjgyMZrxK,0.684,0.607,11,-8.127,...,0.000307,0.1910,0.498,81.001,audio_features,spotify:track:0JXXNGljqupsJaZsgSbMZV,https://api.spotify.com/v1/tracks/0JXXNGljqups...,https://api.spotify.com/v1/audio-analysis/0JXX...,195373,4
2,51pshtuYkgUQnt5huMPbKL,The Spins,82,True,Mac Miller,4LLpKhyESsyAXpc4laK94U,0.794,0.766,9,-3.894,...,0.000000,0.0785,0.550,127.007,audio_features,spotify:track:51pshtuYkgUQnt5huMPbKL,https://api.spotify.com/v1/tracks/51pshtuYkgUQ...,https://api.spotify.com/v1/audio-analysis/51ps...,195873,4
3,1eUGmzzvahJjOSWgDHuRlv,Girl With The Tattoo Enter.lewd,81,False,Miguel,360IAlyVv4PCEVjgyMZrxK,0.372,0.239,6,-11.314,...,0.000175,0.1680,0.127,113.451,audio_features,spotify:track:1eUGmzzvahJjOSWgDHuRlv,https://api.spotify.com/v1/tracks/1eUGmzzvahJj...,https://api.spotify.com/v1/audio-analysis/1eUG...,102813,4
4,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,83,False,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,0.635,0.841,5,-5.379,...,0.000000,0.0622,0.424,109.021,audio_features,spotify:track:7BqBn9nzAq8spo5e7cZ0dJ,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,https://api.spotify.com/v1/audio-analysis/7BqB...,220734,4


In [40]:
df_2010.to_csv('df_2010.csv')

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,27RYrbL6S02LNVhDWVl38b,Happy Xmas (War Is Over) - Remastered 2010,91,False,John Lennon,4x1nvY2FN8jxqAFA0DA02H,0.321,0.640,2,-10.023,...,0.000000,0.7180,0.4140,146.172,audio_features,spotify:track:27RYrbL6S02LNVhDWVl38b,https://api.spotify.com/v1/tracks/27RYrbL6S02L...,https://api.spotify.com/v1/audio-analysis/27RY...,214333,3
1,0JXXNGljqupsJaZsgSbMZV,Sure Thing,87,False,Miguel,360IAlyVv4PCEVjgyMZrxK,0.684,0.607,11,-8.127,...,0.000307,0.1910,0.4980,81.001,audio_features,spotify:track:0JXXNGljqupsJaZsgSbMZV,https://api.spotify.com/v1/tracks/0JXXNGljqups...,https://api.spotify.com/v1/audio-analysis/0JXX...,195373,4
2,51pshtuYkgUQnt5huMPbKL,The Spins,82,True,Mac Miller,4LLpKhyESsyAXpc4laK94U,0.794,0.766,9,-3.894,...,0.000000,0.0785,0.5500,127.007,audio_features,spotify:track:51pshtuYkgUQnt5huMPbKL,https://api.spotify.com/v1/tracks/51pshtuYkgUQ...,https://api.spotify.com/v1/audio-analysis/51ps...,195873,4
3,1eUGmzzvahJjOSWgDHuRlv,Girl With The Tattoo Enter.lewd,81,False,Miguel,360IAlyVv4PCEVjgyMZrxK,0.372,0.239,6,-11.314,...,0.000175,0.1680,0.1270,113.451,audio_features,spotify:track:1eUGmzzvahJjOSWgDHuRlv,https://api.spotify.com/v1/tracks/1eUGmzzvahJj...,https://api.spotify.com/v1/audio-analysis/1eUG...,102813,4
4,7BqBn9nzAq8spo5e7cZ0dJ,Just the Way You Are,83,False,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,0.635,0.841,5,-5.379,...,0.000000,0.0622,0.4240,109.021,audio_features,spotify:track:7BqBn9nzAq8spo5e7cZ0dJ,https://api.spotify.com/v1/tracks/7BqBn9nzAq8s...,https://api.spotify.com/v1/audio-analysis/7BqB...,220734,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,58vvxoXqQicDlUFJHfOuEs,Warsaw - 2010 Remaster,55,False,Joy Division,432R46LaYsJZV2Gmc4jUV5,0.561,0.700,0,-8.820,...,0.000030,0.0723,0.6070,82.857,audio_features,spotify:track:58vvxoXqQicDlUFJHfOuEs,https://api.spotify.com/v1/tracks/58vvxoXqQicD...,https://api.spotify.com/v1/audio-analysis/58vv...,144662,4
996,5mEyCUtI36Jmu2KNQQ4jaw,Girls Like You,52,False,The Naked And Famous,0oeUpvxWsC8bWS6SnpU8b9,0.491,0.481,4,-5.762,...,0.000167,0.1050,0.0546,124.858,audio_features,spotify:track:5mEyCUtI36Jmu2KNQQ4jaw,https://api.spotify.com/v1/tracks/5mEyCUtI36Jm...,https://api.spotify.com/v1/audio-analysis/5mEy...,364493,4
997,6z4LGme7tL991yvpk1gURW,Me Volví A Acordar De Ti,55,False,Los Angeles De Charly,0fdpJLsenUo15X3dPPWJSR,0.673,0.579,2,-4.587,...,0.000000,0.4230,0.7760,145.970,audio_features,spotify:track:6z4LGme7tL991yvpk1gURW,https://api.spotify.com/v1/tracks/6z4LGme7tL99...,https://api.spotify.com/v1/audio-analysis/6z4L...,235293,4
998,72phhQtfwZcRweweRxxkmU,According To You,49,False,Orianthi,0yNy8fi1yBBq526E6mx4Zs,0.619,0.917,5,-2.582,...,0.000012,0.0986,0.5520,131.002,audio_features,spotify:track:72phhQtfwZcRweweRxxkmU,https://api.spotify.com/v1/tracks/72phhQtfwZcR...,https://api.spotify.com/v1/audio-analysis/72ph...,200347,4


In [25]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2009', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2009 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2009.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5Q0Nhxo0l2bP3pNjpGJwV1,Party In The U.S.A.,75,False,Miley Cyrus,5YGY8feqx7naU7z4HrwZM6,0.652,0.698,10,-4.667,...,0.000115,0.0886,0.470,96.021,audio_features,spotify:track:5Q0Nhxo0l2bP3pNjpGJwV1,https://api.spotify.com/v1/tracks/5Q0Nhxo0l2bP...,https://api.spotify.com/v1/audio-analysis/5Q0N...,202067,4
1,3QLjDkgLh9AOEHlhQtDuhs,Best I Ever Had,77,True,Drake,3TVXtAsR1Inumwj472S9r4,0.395,0.922,1,-2.176,...,0.000000,0.1340,0.558,162.361,audio_features,spotify:track:3QLjDkgLh9AOEHlhQtDuhs,https://api.spotify.com/v1/tracks/3QLjDkgLh9AO...,https://api.spotify.com/v1/audio-analysis/3QLj...,257690,4
2,5iSEsR6NKjlC9SrIJkyL3k,Pursuit Of Happiness (Nightmare),74,True,Kid Cudi,0fA0VVWsXO9YnASrzqfmYu,0.630,0.605,0,-8.951,...,0.000093,0.2980,0.266,115.277,audio_features,spotify:track:5iSEsR6NKjlC9SrIJkyL3k,https://api.spotify.com/v1/tracks/5iSEsR6NKjlC...,https://api.spotify.com/v1/audio-analysis/5iSE...,295293,4
3,3Pzh926pXggbMe2ZpXyMV7,Ain't No Rest for the Wicked,75,False,Cage The Elephant,26T3LtbuGT1Fu9m0eRq5X3,0.636,0.849,0,-7.075,...,0.000000,0.3720,0.917,156.036,audio_features,spotify:track:3Pzh926pXggbMe2ZpXyMV7,https://api.spotify.com/v1/tracks/3Pzh926pXggb...,https://api.spotify.com/v1/audio-analysis/3Pzh...,175493,4
4,2igwFfvr1OAGX9SKDCPBwO,Empire State Of Mind,83,True,JAY-Z,3nFkdlSjzX9mRTtwJOzDYB,0.491,0.956,11,-1.538,...,0.000000,0.4600,0.811,173.585,audio_features,spotify:track:2igwFfvr1OAGX9SKDCPBwO,https://api.spotify.com/v1/tracks/2igwFfvr1OAG...,https://api.spotify.com/v1/audio-analysis/2igw...,276920,4


In [41]:
df_2009.to_csv('df_2009.csv')

In [43]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2008', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2008 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2008.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,4EWCNWgDS8707fNSZ1oaA5,Heartless,85,False,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,0.790,0.6470,10,-5.983,...,0.000000,0.248,0.6540,87.999,audio_features,spotify:track:4EWCNWgDS8707fNSZ1oaA5,https://api.spotify.com/v1/tracks/4EWCNWgDS870...,https://api.spotify.com/v1/audio-analysis/4EWC...,211000,4
1,1mea3bSkSGXuIRvnydlB5b,Viva La Vida,86,False,Coldplay,4gzpq5DPGxSnKTe4SA8HAU,0.486,0.6170,5,-7.115,...,0.000003,0.109,0.4170,138.015,audio_features,spotify:track:1mea3bSkSGXuIRvnydlB5b,https://api.spotify.com/v1/tracks/1mea3bSkSGXu...,https://api.spotify.com/v1/audio-analysis/1mea...,242373,4
2,6TfBA04WJ3X1d1wXhaCFVT,"You're Gonna Go Far, Kid",82,True,The Offspring,5LfGQac0EIXyAN8aUwmNAQ,0.550,0.9170,0,-3.159,...,0.000000,0.197,0.6010,126.115,audio_features,spotify:track:6TfBA04WJ3X1d1wXhaCFVT,https://api.spotify.com/v1/tracks/6TfBA04WJ3X1...,https://api.spotify.com/v1/audio-analysis/6TfB...,177827,4
3,2VsCE6ui7N4IRzGIGT7Di8,O Christmas Tree,76,False,Tony Bennett,2lolQgalUvZDfp5vvVtTYV,0.571,0.0982,6,-14.340,...,0.000002,0.109,0.0857,74.271,audio_features,spotify:track:2VsCE6ui7N4IRzGIGT7Di8,https://api.spotify.com/v1/tracks/2VsCE6ui7N4I...,https://api.spotify.com/v1/audio-analysis/2VsC...,207253,4
4,0LQtEJt7x0s6knb6RKdRYc,Chicken Fried,77,False,Zac Brown Band,6yJCxee7QumYr820xdIsjo,0.566,0.7130,6,-4.250,...,0.000000,0.114,0.8050,169.861,audio_features,spotify:track:0LQtEJt7x0s6knb6RKdRYc,https://api.spotify.com/v1/tracks/0LQtEJt7x0s6...,https://api.spotify.com/v1/audio-analysis/0LQt...,238147,4


In [44]:
df_2008.to_csv('df_2008.csv')

In [45]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2007', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2007 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2007.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,58ge6dfP91o9oXMzq3XkIS,505,82,False,Arctic Monkeys,7Ln80lUS6He07XvHI8qqHH,0.520,0.852,0,-5.866,...,0.000058,0.0733,0.234,140.267,audio_features,spotify:track:58ge6dfP91o9oXMzq3XkIS,https://api.spotify.com/v1/tracks/58ge6dfP91o9...,https://api.spotify.com/v1/audio-analysis/58ge...,253587,4
1,5TRPicyLGbAF2LGBFbHGvO,Flashing Lights,85,True,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,0.639,0.628,6,-7.578,...,0.000000,0.3860,0.430,90.482,audio_features,spotify:track:5TRPicyLGbAF2LGBFbHGvO,https://api.spotify.com/v1/tracks/5TRPicyLGbAF...,https://api.spotify.com/v1/audio-analysis/5TRP...,237507,4
2,7rbECVPkY5UODxoOUVKZnA,I Wonder,82,True,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,0.542,0.466,0,-8.665,...,0.000445,0.1250,0.124,191.385,audio_features,spotify:track:7rbECVPkY5UODxoOUVKZnA,https://api.spotify.com/v1/tracks/7rbECVPkY5UO...,https://api.spotify.com/v1/audio-analysis/7rbE...,243440,4
3,6SpLc7EXZIPpy0sVko0aoU,Misery Business,76,False,Paramore,74XFHRwlV6OrjEM0A2NCMF,0.517,0.906,1,-3.677,...,0.000009,0.1130,0.731,172.977,audio_features,spotify:track:6SpLc7EXZIPpy0sVko0aoU,https://api.spotify.com/v1/tracks/6SpLc7EXZIPp...,https://api.spotify.com/v1/audio-analysis/6SpL...,211520,4
4,4iz9lGMjU1lXS51oPmUmTe,Homecoming,80,True,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,0.667,0.747,1,-7.059,...,0.000000,0.1150,0.918,86.917,audio_features,spotify:track:4iz9lGMjU1lXS51oPmUmTe,https://api.spotify.com/v1/tracks/4iz9lGMjU1lX...,https://api.spotify.com/v1/audio-analysis/4iz9...,203493,4


In [46]:
df_2007.to_csv('df_2007.csv')

In [47]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2006', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2006 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2006.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,5aDoUmxBsbdpS16alksb5Z,Do You Hear What I Hear? - Remastered 2006,83,False,Bing Crosby,6ZjFtWeHP9XN7FeKSUe80S,0.566,0.246,10,-15.499,...,0.000001,0.1950,0.412,101.529,audio_features,spotify:track:5aDoUmxBsbdpS16alksb5Z,https://api.spotify.com/v1/tracks/5aDoUmxBsbdp...,https://api.spotify.com/v1/audio-analysis/5aDo...,165667,4
1,6wn2nmFn3wDuiMldRiuRuL,Last Christmas - Remastered,85,False,Wham!,5lpH0xAS4fVfLkACg9DAuM,0.757,0.618,11,-7.964,...,0.000000,0.2210,0.861,106.856,audio_features,spotify:track:6wn2nmFn3wDuiMldRiuRuL,https://api.spotify.com/v1/tracks/6wn2nmFn3wDu...,https://api.spotify.com/v1/audio-analysis/6wn2...,267160,4
2,2IuUMx3uxxJAHcH41aYtn0,Winter Wonderland - Remastered 2006,80,False,Bing Crosby,6ZjFtWeHP9XN7FeKSUe80S,0.592,0.230,10,-13.603,...,0.000000,0.1200,0.537,126.655,audio_features,spotify:track:2IuUMx3uxxJAHcH41aYtn0,https://api.spotify.com/v1/tracks/2IuUMx3uxxJA...,https://api.spotify.com/v1/audio-analysis/2IuU...,144253,4
3,2gam98EZKrF9XuOkU13ApN,Promiscuous,83,False,Nelly Furtado,2jw70GZXlAI8QzWeY2bgRc,0.808,0.970,10,-6.098,...,0.000061,0.1540,0.868,114.328,audio_features,spotify:track:2gam98EZKrF9XuOkU13ApN,https://api.spotify.com/v1/tracks/2gam98EZKrF9...,https://api.spotify.com/v1/audio-analysis/2gam...,242293,4
4,35MqcEIVZ7svbBpRt4N30g,Frosty The Snowman - Remastered 2006,78,False,Bing Crosby,6ZjFtWeHP9XN7FeKSUe80S,0.519,0.368,7,-11.622,...,0.000000,0.0859,0.854,160.370,audio_features,spotify:track:35MqcEIVZ7svbBpRt4N30g,https://api.spotify.com/v1/tracks/35MqcEIVZ7sv...,https://api.spotify.com/v1/audio-analysis/35Mq...,133413,4


In [48]:
df_2006.to_csv('df_2006.csv')

In [49]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2005', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2005 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2005.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,0d28khcov6AiegSCpG5TuT,Feel Good Inc.,84,False,Gorillaz,3AA28KZvwAUcZuOKwyblJQ,0.818,0.705,6,-6.679,...,0.00233,0.6130,0.772,138.559,audio_features,spotify:track:0d28khcov6AiegSCpG5TuT,https://api.spotify.com/v1/tracks/0d28khcov6Ai...,https://api.spotify.com/v1/audio-analysis/0d28...,222640,4
1,1PS1QMdUqOal0ai3Gt7sDQ,Gold Digger,80,True,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,0.629,0.696,1,-5.572,...,0.00000,0.0554,0.623,93.034,audio_features,spotify:track:1PS1QMdUqOal0ai3Gt7sDQ,https://api.spotify.com/v1/tracks/1PS1QMdUqOal...,https://api.spotify.com/v1/audio-analysis/1PS1...,207627,4
2,2TfSHkHiFO4gRztVIkggkE,"Sugar, We're Goin Down",78,False,Fall Out Boy,4UXqAaa6dQYAk18Lv7PEgX,0.494,0.842,7,-4.784,...,0.00000,0.1220,0.574,162.011,audio_features,spotify:track:2TfSHkHiFO4gRztVIkggkE,https://api.spotify.com/v1/tracks/2TfSHkHiFO4g...,https://api.spotify.com/v1/audio-analysis/2TfS...,229093,4
3,0ZUo4YjG4saFnEJhdWp9Bt,Before He Cheats,78,False,Carrie Underwood,4xFUf1FHVy696Q1JQZMTRj,0.519,0.749,6,-3.318,...,0.00000,0.1190,0.290,147.905,audio_features,spotify:track:0ZUo4YjG4saFnEJhdWp9Bt,https://api.spotify.com/v1/tracks/0ZUo4YjG4saF...,https://api.spotify.com/v1/audio-analysis/0ZUo...,199947,4
4,4bPQs0PHn4xbipzdPfn6du,I Write Sins Not Tragedies,74,False,Panic! At The Disco,20JZFwl6HVl6yg8a4H3ZqK,0.569,0.830,9,-4.106,...,0.00000,0.1140,0.609,170.094,audio_features,spotify:track:4bPQs0PHn4xbipzdPfn6du,https://api.spotify.com/v1/tracks/4bPQs0PHn4xb...,https://api.spotify.com/v1/audio-analysis/4bPQ...,186635,4


In [50]:
df_2005.to_csv('df_2005.csv')

In [51]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2004', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2004 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2004.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,0
0,561jH07mF1jHuk7KlaeF0s,Mockingbird,93.0,True,Eminem,7dGJo4pcD2V6oG8kP0tJRR,0.637,0.678,0.0,-3.798,...,0.1560,0.254,84.039,audio_features,spotify:track:561jH07mF1jHuk7KlaeF0s,https://api.spotify.com/v1/tracks/561jH07mF1jH...,https://api.spotify.com/v1/audio-analysis/561j...,250760.0,4.0,NaN
1,003vvx7Niy0yvhvHt4a68B,Mr. Brightside,86.0,False,The Killers,0C0XlULifJtAgn6ZNCW2eu,0.352,0.911,1.0,-5.230,...,0.0995,0.236,148.033,audio_features,spotify:track:003vvx7Niy0yvhvHt4a68B,https://api.spotify.com/v1/tracks/003vvx7Niy0y...,https://api.spotify.com/v1/audio-analysis/003v...,222973.0,4.0,NaN
2,5SkRLpaGtvYPhw02vZhQQ9,All Falls Down,80.0,True,Kanye West,5K4W6rqBFWDnAN6FQUkS6x,0.657,0.734,8.0,-4.832,...,0.1390,0.434,91.030,audio_features,spotify:track:5SkRLpaGtvYPhw02vZhQQ9,https://api.spotify.com/v1/tracks/5SkRLpaGtvYP...,https://api.spotify.com/v1/audio-analysis/5SkR...,223507.0,4.0,NaN
3,5rb9QrpfcKFHM1EUbSIurX,Yeah! (feat. Lil Jon & Ludacris),82.0,False,Usher,23zg3TcAtWQy7J6upgbUnj,0.894,0.791,2.0,-4.699,...,0.0388,0.583,105.018,audio_features,spotify:track:5rb9QrpfcKFHM1EUbSIurX,https://api.spotify.com/v1/tracks/5rb9QrpfcKFH...,https://api.spotify.com/v1/audio-analysis/5rb9...,250373.0,4.0,NaN
4,0ll8uFnc0nANY35E0Lfxvg,Somewhere Only We Know,77.0,False,Keane,53A0W3U0s8diEn9RhXQhVz,0.453,0.655,9.0,-4.220,...,0.0849,0.368,172.012,audio_features,spotify:track:0ll8uFnc0nANY35E0Lfxvg,https://api.spotify.com/v1/tracks/0ll8uFnc0nAN...,https://api.spotify.com/v1/audio-analysis/0ll8...,236160.0,4.0,NaN


In [52]:
df_2004.to_csv('df_2004.csv')

In [53]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2003', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2003 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2003.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3dPQuX8Gs42Y7b454ybpMR,Seven Nation Army,84,False,The White Stripes,4F84IBURUo98rz4r61KF70,0.743,0.446,4,-7.807,...,0.118000,0.3370,0.278,123.596,audio_features,spotify:track:3dPQuX8Gs42Y7b454ybpMR,https://api.spotify.com/v1/tracks/3dPQuX8Gs42Y...,https://api.spotify.com/v1/audio-analysis/3dPQ...,232107,4
1,4RY96Asd9IefaL3X4LOLZ8,In Da Club,76,True,50 Cent,3q7HBObVc0L8jNeTe5Gofh,0.902,0.720,6,-2.776,...,0.000000,0.0749,0.805,90.059,audio_features,spotify:track:4RY96Asd9IefaL3X4LOLZ8,https://api.spotify.com/v1/tracks/4RY96Asd9Ief...,https://api.spotify.com/v1/audio-analysis/4RY9...,193467,4
2,2PpruBYCo4H7WOBJ7Q2EwM,Hey Ya!,81,False,Outkast,1G9G7WwrXka3Z1r7aIDjI7,0.727,0.974,4,-2.261,...,0.000532,0.1740,0.965,79.526,audio_features,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,https://api.spotify.com/v1/tracks/2PpruBYCo4H7...,https://api.spotify.com/v1/audio-analysis/2Ppr...,235213,4
3,2nLtzopw4rPReszdYBJU6h,Numb,84,False,Linkin Park,6XyY86QOPPrYVGvF9ch6wz,0.496,0.863,9,-4.153,...,0.000000,0.6390,0.243,110.018,audio_features,spotify:track:2nLtzopw4rPReszdYBJU6h,https://api.spotify.com/v1/tracks/2nLtzopw4rPR...,https://api.spotify.com/v1/audio-analysis/2nLt...,185587,4
4,69Ej1xrGjOcHvIMtMKxK0G,Dile,84,False,Don Omar,33ScadVnbm2X8kkUqOkC6Z,0.817,0.714,4,-7.501,...,0.000132,0.0420,0.714,94.001,audio_features,spotify:track:69Ej1xrGjOcHvIMtMKxK0G,https://api.spotify.com/v1/tracks/69Ej1xrGjOcH...,https://api.spotify.com/v1/audio-analysis/69Ej...,204600,4


In [54]:
df_2003.to_csv('df_2003.csv')

In [55]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2002', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2002 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2002.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,0oPdaY4dXtc3ZsaG17V972,Feliz Navidad,87,False,José Feliciano,7K78lVZ8XzkjfRSI7570FF,0.513,0.831,2,-9.004,...,0.0,0.3360,0.963,148.837,audio_features,spotify:track:0oPdaY4dXtc3ZsaG17V972,https://api.spotify.com/v1/tracks/0oPdaY4dXtc3...,https://api.spotify.com/v1/audio-analysis/0oPd...,182067,4
1,7lQ8MOhq6IN2w8EYcFNSUk,Without Me,89,True,Eminem,7dGJo4pcD2V6oG8kP0tJRR,0.908,0.669,7,-2.827,...,0.0,0.2370,0.662,112.238,audio_features,spotify:track:7lQ8MOhq6IN2w8EYcFNSUk,https://api.spotify.com/v1/tracks/7lQ8MOhq6IN2...,https://api.spotify.com/v1/audio-analysis/7lQ8...,290320,4
2,4xkOaSrkexMciUUogZKVTS,Till I Collapse,87,True,Eminem,7dGJo4pcD2V6oG8kP0tJRR,0.548,0.847,1,-3.237,...,0.0,0.0816,0.100,171.447,audio_features,spotify:track:4xkOaSrkexMciUUogZKVTS,https://api.spotify.com/v1/tracks/4xkOaSrkexMc...,https://api.spotify.com/v1/audio-analysis/4xkO...,297787,4
3,4woTEX1wYOTGDqNXuavlRC,Superman,88,True,Eminem,7dGJo4pcD2V6oG8kP0tJRR,0.802,0.755,4,-3.138,...,0.0,0.2950,0.580,130.027,audio_features,spotify:track:4woTEX1wYOTGDqNXuavlRC,https://api.spotify.com/v1/tracks/4woTEX1wYOTG...,https://api.spotify.com/v1/audio-analysis/4woT...,350320,4
4,1foCxQtxBweJtZmdxhEHVO,Santa Baby (with Henri René & His Orchestra),79,False,Eartha Kitt,1AwO9pWEBSBoWdEZu28XDC,0.700,0.139,8,-12.346,...,0.0,0.1850,0.490,84.485,audio_features,spotify:track:1foCxQtxBweJtZmdxhEHVO,https://api.spotify.com/v1/tracks/1foCxQtxBweJ...,https://api.spotify.com/v1/audio-analysis/1foC...,206067,4


In [56]:
df_2002.to_csv('df_2002.csv')

In [57]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2001', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2001 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2001.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,6GG73Jik4jUlQCkKg9JuGO,The Middle,81,False,Jimmy Eat World,3Ayl7mCk0nScecqOzvNp6s,0.643,0.849,2,-5.428,...,0.00000,0.058,0.903,162.152,audio_features,spotify:track:6GG73Jik4jUlQCkKg9JuGO,https://api.spotify.com/v1/tracks/6GG73Jik4jUl...,https://api.spotify.com/v1/audio-analysis/6GG7...,165853,4
1,2DlHlPMa4M17kufBvI2lEN,Chop Suey!,84,False,System Of A Down,5eAWCfyUhZtHHtBdNk56l1,0.417,0.934,7,-3.908,...,0.00150,0.132,0.287,127.066,audio_features,spotify:track:2DlHlPMa4M17kufBvI2lEN,https://api.spotify.com/v1/tracks/2DlHlPMa4M17...,https://api.spotify.com/v1/audio-analysis/2DlH...,210240,4
2,2GapxG7BxK55ihQRAlR39e,(There's No Place Like) Home for the Holidays ...,77,False,Perry Como,5v8jlSmAQfrkTjAlpUfWtu,0.532,0.401,5,-10.629,...,0.00000,0.222,0.450,143.823,audio_features,spotify:track:2GapxG7BxK55ihQRAlR39e,https://api.spotify.com/v1/tracks/2GapxG7BxK55...,https://api.spotify.com/v1/audio-analysis/2Gap...,178293,4
3,5W8YXBz9MTIDyrpYaCg2Ky,Last Resort,81,True,Papa Roach,4RddZ3iHvSpGV4dvATac9X,0.589,0.890,4,-3.719,...,0.00082,0.201,0.692,90.598,audio_features,spotify:track:5W8YXBz9MTIDyrpYaCg2Ky,https://api.spotify.com/v1/tracks/5W8YXBz9MTID...,https://api.spotify.com/v1/audio-analysis/5W8Y...,199907,4
4,1Kcy7qSbf6un30zIkxGVew,Here We Come a-Caroling / We Wish You a Merry ...,75,False,Perry Como,5v8jlSmAQfrkTjAlpUfWtu,0.573,0.246,8,-11.543,...,0.00000,0.126,0.695,95.413,audio_features,spotify:track:1Kcy7qSbf6un30zIkxGVew,https://api.spotify.com/v1/tracks/1Kcy7qSbf6un...,https://api.spotify.com/v1/audio-analysis/1Kcy...,98267,3


In [58]:
df_2001
df_2001.to_csv('df_2001.csv')

In [59]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2000', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_2000 = pd.merge(track_df, features_df, how='outer', on = 'id')
df_2000.head()

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,0YWUHZPJVg4iujddsJDwhM,My Only Wish (This Year),87,False,Britney Spears,26dSoYclwsYLMAKD3tpOr4,0.669,0.843,0,-3.766,...,0.000000,0.1950,0.702,147.080,audio_features,spotify:track:0YWUHZPJVg4iujddsJDwhM,https://api.spotify.com/v1/tracks/0YWUHZPJVg4i...,https://api.spotify.com/v1/audio-analysis/0YWU...,255400,4
1,7D0RhFcb3CrfPuTJ0obrod,Sparks,86,False,Coldplay,4gzpq5DPGxSnKTe4SA8HAU,0.371,0.268,1,-10.506,...,0.051700,0.1040,0.165,102.617,audio_features,spotify:track:7D0RhFcb3CrfPuTJ0obrod,https://api.spotify.com/v1/tracks/7D0RhFcb3Crf...,https://api.spotify.com/v1/audio-analysis/7D0R...,227093,3
2,3yfqSUWxFvZELEM4PmlwIR,The Real Slim Shady,88,True,Eminem,7dGJo4pcD2V6oG8kP0tJRR,0.949,0.661,5,-4.244,...,0.000000,0.0454,0.760,104.504,audio_features,spotify:track:3yfqSUWxFvZELEM4PmlwIR,https://api.spotify.com/v1/tracks/3yfqSUWxFvZE...,https://api.spotify.com/v1/audio-analysis/3yfq...,284200,4
3,3AJwUDP919kvQ9QcozQPxg,Yellow,89,False,Coldplay,4gzpq5DPGxSnKTe4SA8HAU,0.429,0.661,11,-7.227,...,0.000121,0.2340,0.285,173.372,audio_features,spotify:track:3AJwUDP919kvQ9QcozQPxg,https://api.spotify.com/v1/tracks/3AJwUDP919kv...,https://api.spotify.com/v1/audio-analysis/3AJw...,266773,4
4,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,82,True,Outkast,1G9G7WwrXka3Z1r7aIDjI7,0.843,0.806,4,-5.946,...,0.000000,0.0771,0.613,94.948,audio_features,spotify:track:0I3q5fE6wg7LIfHGngUTnV,https://api.spotify.com/v1/tracks/0I3q5fE6wg7L...,https://api.spotify.com/v1/audio-analysis/0I3q...,270507,4


In [60]:
df_2000
df_2000.to_csv('df_2000.csv')

In [66]:
artist_name = []
track_name = []
track_popularity = []
track_explicity =[]
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:1999', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])
        track_explicity.append(t['explicit'])

track_df = pd.DataFrame({'id' : track_id, 'track_name' : track_name,  'track_popularity' : track_popularity, 'explicit':track_explicity, 'artist_name' : artist_name, 'artist_id' : artist_id})

features_df = pd.DataFrame()
for id in track_df['id'].items():
    track_id = id[1]
    audio_features = sp.audio_features(track_id)
    local_features = pd.DataFrame(audio_features, index=[1])
    features_df = pd.concat([features_df, local_features])
    
df_1999 = pd.merge(track_df, features_df, how='outer', on = 'id')

df_1999

,id,track_name,track_popularity,explicit,artist_name,artist_id,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2pXpURmn6zC5ZYDMms6fwa,It's Beginning to Look a Lot Like Christmas (w...,89,False,Perry Como,5v8jlSmAQfrkTjAlpUfWtu,0.724,0.276,7,-12.250,...,0.000000,0.1160,0.7180,112.874,audio_features,spotify:track:2pXpURmn6zC5ZYDMms6fwa,https://api.spotify.com/v1/tracks/2pXpURmn6zC5...,https://api.spotify.com/v1/audio-analysis/2pXp...,155933,4
1,2m1hi0nfMR9vdGC8UcrnwU,All The Small Things,82,False,blink-182,6FBDaR13swtiWwGhX1WQsP,0.434,0.897,0,-4.918,...,0.000000,0.6120,0.6840,148.726,audio_features,spotify:track:2m1hi0nfMR9vdGC8UcrnwU,https://api.spotify.com/v1/tracks/2m1hi0nfMR9v...,https://api.spotify.com/v1/audio-analysis/2m1h...,167067,4
2,503OTo2dSqe7qk76rgsbep,Still D.R.E.,84,True,Dr. Dre,6DPYiyq5kWVQS4RGwxzPC7,0.816,0.775,11,-3.323,...,0.000310,0.0543,0.5270,93.431,audio_features,spotify:track:503OTo2dSqe7qk76rgsbep,https://api.spotify.com/v1/tracks/503OTo2dSqe7...,https://api.spotify.com/v1/audio-analysis/503O...,270587,4
3,7iXF2W9vKmDoGAhlHdpyIa,Forgot About Dre,81,True,Dr. Dre,6DPYiyq5kWVQS4RGwxzPC7,0.924,0.740,8,-1.299,...,0.000000,0.1630,0.6210,133.974,audio_features,spotify:track:7iXF2W9vKmDoGAhlHdpyIa,https://api.spotify.com/v1/tracks/7iXF2W9vKmDo...,https://api.spotify.com/v1/audio-analysis/7iXF...,222293,4
4,1KGi9sZVMeszgZOWivFpxs,No Scrubs,80,False,TLC,0TImkz4nPqjegtVSMZnMRq,0.743,0.675,8,-4.267,...,0.000717,0.0685,0.5900,92.909,audio_features,spotify:track:1KGi9sZVMeszgZOWivFpxs,https://api.spotify.com/v1/tracks/1KGi9sZVMesz...,https://api.spotify.com/v1/audio-analysis/1KGi...,214400,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5M67471d2HYB8z8f0VZQhv,Gimme Some Lovin',44,False,G. Love & Special Sauce,74fkl73HDlCXw0l6cemB89,0.576,0.276,0,-15.105,...,0.000118,0.0938,0.8670,90.965,audio_features,spotify:track:5M67471d2HYB8z8f0VZQhv,https://api.spotify.com/v1/tracks/5M67471d2HYB...,https://api.spotify.com/v1/audio-analysis/5M67...,143000,4
996,3X1ckVnSNleVzJLAvKKik6,White Christmas - 1959 Version,45,False,Perry Como,5v8jlSmAQfrkTjAlpUfWtu,0.292,0.177,0,-14.765,...,0.000000,0.0923,0.1810,95.269,audio_features,spotify:track:3X1ckVnSNleVzJLAvKKik6,https://api.spotify.com/v1/tracks/3X1ckVnSNleV...,https://api.spotify.com/v1/audio-analysis/3X1c...,190840,4
997,2udRqNX3z2OjdqR01ebDwU,No Dices Mas,61,False,Mœnia,3QmmtMrEf7aQrsd1VtejAV,0.707,0.716,1,-6.939,...,0.000929,0.0590,0.5970,127.017,audio_features,spotify:track:2udRqNX3z2OjdqR01ebDwU,https://api.spotify.com/v1/tracks/2udRqNX3z2Oj...,https://api.spotify.com/v1/audio-analysis/2udR...,259800,4
998,7lX2zznCc07XDOe9Kt9st7,Eastside Story,45,False,Trevor Duncan,6IXtUyCfa1MhyE5OkVNDrg,0.164,0.355,6,-8.874,...,0.736000,0.1420,0.0787,133.377,audio_features,spotify:track:7lX2zznCc07XDOe9Kt9st7,https://api.spotify.com/v1/tracks/7lX2zznCc07X...,https://api.spotify.com/v1/audio-analysis/7lX2...,87400,4


In [67]:
df_1999.to_csv('df_1999.csv')
